In [1]:
import urllib.request as ur
import re
import json
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
filename = "professors.txt"
profs =  open(filename, "r").read()
profs_json = json.loads(profs)
profiles = profs_json['response']['docs']
print(profiles[0])

{'teacherlastname_t': 'Aamari', 'pk_id': 2325936, 'schoolid_s': '1079', 'averageratingscore_rf': 4.2, 'total_number_of_ratings_i': 6, 'teacherfirstname_t': 'Eddie'}


In [9]:
def getIds(d):
    ids = []
    for p in d:
        ids.append(p['pk_id'])
    return ids


In [26]:
ids = getIds(profiles)


2325936


### Run me, excercise caution

In [10]:
ids = getIds(profiles)
comments = []
for i in ids:
    print(i)
    comments.append(getProfData(i))
data = pd.concat(comments)

2325936


NameError: name 'getProfData' is not defined

In [49]:
data.to_csv("comments.csv", encoding='utf-8')

In [51]:
data.shape

(42132, 31)

In [7]:
fake_hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}

In [8]:
def getHtml(prof_id):
     
    url = 'http://www.ratemyprofessors.com/ShowRatings.jsp?tid='+str(prof_id)
    req = ur.Request(url, headers=fake_hdr)
    
    try:
        html = ur.urlopen(req)
    except ur.HTTPError as e:
        print(e.fp.read())
        
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [9]:
getHtml(2325936)


<!DOCTYPE html>

<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js"> <!--<![endif]-->
<head>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="F8gUuqzRvOrAlhaGyP7aAuMs_Se8zK-98Ai2sNsIZEo" name="google-site-verification">
<meta content="hk1NnSbYuDC0Sppgbf7YIT-VxUiRbOVRdtqA4AmkGzM" name="google-site-verification"/>
<meta content="CKMC_IwvKoVbX1U8x1A9yzKABsSlop6qxfuDzwfV7Qs" name="google-site-verification"/> <!--  relaunch qa -->
<meta content="1D_3ZfAdMu4Tki8pFRj68YAqYot-paTOoDVzCTIJZZI" name="google-site-verification"/> <!--  relaunch rmp -->
<meta content="14147781149-i5ph5oqooelp3k0qfrb3jkk6vs4528ha.apps.googleusercontent.com" name="google-signin-client_id"/>
<script src="https://use.fontawesome.com/fbed9f71e3.js"></script>
<link href="/assets/application-b6b4b77

In [27]:
def getName(prof_id):
    soup = getHtml(prof_id) 
    first_name_blocks = soup.find_all('span', class_='pfname')
    last_name_blocks = soup.find_all('span', class_='plname')
    if(len(first_name_blocks) == 0 and len(last_name_blocks) ==0):
        return "Na"
    fname = re.sub(r'(\s)+', '', first_name_blocks[0].get_text())
    lname = re.sub(r'(\s)+', '', last_name_blocks[0].get_text())
    return fname + " " + lname

print(getName(2325936))

Eddie Aamari


In [17]:
def getScore(prof_id):   
    soup = getHtml(prof_id)
    score = float(soup.find_all(class_='grade')[0].get_text())
    return int(score)

In [18]:
def getComments(prof_id):
    page_id=1
    pages = []
    while True:
        url = 'http://www.ratemyprofessors.com/paginate/professors/ratings?tid='+str(prof_id)+'&filter=&courseCode=&page='+str(page_id)
        req = ur.Request(url, headers=fake_hdr)
        try:
            reviews = ur.urlopen(req)
        except ur.HTTPError as e:
            print(e.fp.read()) 
        rt = reviews.read().decode('utf-8')
        r_json = json.loads(rt)
        page_of_comments = pd.DataFrame.from_dict(r_json['ratings'], orient='columns')
        pages.append(page_of_comments)
        print (int(r_json['remaining']))
        if(int(r_json['remaining']) == 0):
            break
        else:
            page_id += 1
    return pages

In [19]:
def getProfData(prof_id):
    pages = getComments(prof_id)
    all_pages = pd.concat(pages)
    name = getName(prof_id)
    all_pages.insert(0, 'pName', name)
    return all_pages

In [94]:
data = pd.concat(pages)

In [95]:
ids = 
for id in Ids

,attendance,clarityColor,easyColor,helpColor,helpCount,id,notHelpCount,onlineClass,quality,rClarity,...,rStatus,rTextBookUse,rWouldTakeAgain,sId,takenForCredit,teacher,teacherGrade,teacherRatingTags,unUsefulGrouping,usefulGrouping
0,Mandatory,good,good,good,0,30832942,0,,awesome,5,...,1,Yes,Yes,877,Yes,None,N/A,"[Amazing lectures, Hilarious, Participation ma...",people,people
1,Mandatory,good,average,good,0,30798524,0,,awesome,5,...,1,Yes,Yes,877,Yes,None,B,"[Respected, Participation matters, Amazing lec...",people,people
2,Mandatory,good,good,good,0,30744671,0,,awesome,5,...,1,Yes,Yes,877,Yes,None,N/A,"[LECTURE HEAVY, Amazing lectures, Participatio...",people,people
3,Mandatory,good,average,good,0,30613385,0,,awesome,5,...,1,No,Yes,877,Yes,None,A,"[Participation matters, Amazing lectures, Hila...",people,people
4,Mandatory,good,average,good,0,30538773,0,,awesome,5,...,1,Yes,Yes,877,Yes,None,A,"[Respected, Participation matters]",people,people
5,Not Mandatory,good,average,good,0,30459521,0,,good,4,...,1,No,Yes,877,Yes,None,N/A,"[Clear grading criteria, LECTURE HEAVY, Amazin...",people,people
6,Mandatory,good,average,good,0,30317948,0,online,good,4,...,1,Yes,Yes,877,Yes,None,A-,"[Participation matters, Amazing lectures]",people,people
7,N/A,good,poor,good,0,30308087,0,online,awesome,5,...,1,Yes,Yes,877,Yes,None,A+,"[Get ready to read, GRADED BY FEW THINGS, Clea...",people,people
8,Mandatory,good,average,good,0,30266349,0,,awesome,5,...,1,Yes,Yes,877,Yes,None,N/A,"[Amazing lectures, Participation matters, Clea...",people,people
9,Not Mandatory,good,average,good,0,30076757,0,,awesome,5,...,1,Yes,Yes,877,Yes,None,A,"[LOTS OF HOMEWORK, Amazing lectures, Participa...",people,people


In [43]:
data = pd.DataFrame.from_dict(r_json['ratings'], orient='columns')
data['name'] = "John"
data

,attendance,clarityColor,easyColor,helpColor,helpCount,id,notHelpCount,onlineClass,quality,rClarity,...,rTextBookUse,rWouldTakeAgain,sId,takenForCredit,teacher,teacherGrade,teacherRatingTags,unUsefulGrouping,usefulGrouping,name
0,Mandatory,good,good,good,0,30832942,0,,awesome,5,...,Yes,Yes,877,Yes,None,N/A,"[Participation matters, Hilarious, Amazing lec...",people,people,John
1,Mandatory,good,average,good,0,30798524,0,,awesome,5,...,Yes,Yes,877,Yes,None,B,"[Participation matters, Respected, Amazing lec...",people,people,John
2,Mandatory,good,good,good,0,30744671,0,,awesome,5,...,Yes,Yes,877,Yes,None,N/A,"[LECTURE HEAVY, Amazing lectures, Participatio...",people,people,John
3,Mandatory,good,average,good,0,30613385,0,,awesome,5,...,No,Yes,877,Yes,None,A,"[Hilarious, Amazing lectures, Participation ma...",people,people,John
4,Mandatory,good,average,good,0,30538773,0,,awesome,5,...,Yes,Yes,877,Yes,None,A,"[Participation matters, Respected]",people,people,John
5,Not Mandatory,good,average,good,0,30459521,0,,good,4,...,No,Yes,877,Yes,None,N/A,"[LECTURE HEAVY, Clear grading criteria, Amazin...",people,people,John
6,Mandatory,good,average,good,0,30317948,0,online,good,4,...,Yes,Yes,877,Yes,None,A-,"[Participation matters, Amazing lectures]",people,people,John
7,N/A,good,poor,good,0,30308087,0,online,awesome,5,...,Yes,Yes,877,Yes,None,A+,"[Clear grading criteria, Get ready to read, GR...",people,people,John
8,Mandatory,good,average,good,0,30266349,0,,awesome,5,...,Yes,Yes,877,Yes,None,N/A,"[Clear grading criteria, Amazing lectures, Par...",people,people,John
9,Not Mandatory,good,average,good,0,30076757,0,,awesome,5,...,Yes,Yes,877,Yes,None,A,"[Participation matters, LOTS OF HOMEWORK, Amaz...",people,people,John


In [54]:
print(r_json['remaining'])

0
